In [19]:
import pandas as pd

# Beispielhafte Struktur für die Kalkulationstabelle mit realistischeren Entwicklungskosten
kostenarten = [
    ('Entwicklungskosten', [
        ('Softwareentwicklung (App für Endanwender)', 75000.0),  
        ('Softwareentwicklung (App/Dashboard für Entsorger)', 50000.0),
        ('Technische Entwicklung LoRaWAN (Integration, Kommunikation)', 15000.0),  # LoRaWAN Entwicklungskosten; vllt kicken wir es raus, um den Preis zu hübschen
        ('Sonstige Entwicklungskosten (z.B. Testing, Debugging, Schnittstellen, UI)', 10000.0),
    ]),

    ('Materialkosten', [
        ('240L Mülltonne (Basis)', 40.0),
        ('Arduino NodeMCU', 6.0),
        ('Ultraschall-Sensor (HCSR04)', 1.5),
        ('Gewichtssensor', 4.0),
        ('Temperatursensor', 2.0),
        ('GPS-Modul', 8.0),
        ('RFID-Reader', 3.0),
        ('LoRaWAN-Modul', 10.0),
        ('Solarpanel', 10.0),
        ('Akku', 5.0),
        ('Gehäuse', 15.0),
        ('Montagekosten', 25.0),  # Montagekosten gehören zu den Materialkosten
    ]),

    ('Betriebskosten', [
        ('LoRaWAN-Betriebskosten (jährlich, geschätzt)', 30.0),
        ('Sonstiges (z.B. Verpackung, Logistik)', 15.0)  # Zuordnung zu Betriebskosten überprüfen
    ])
]

# Zusammenführen in einen DataFrame -> ist einfach nur eine Liste der obigen Einträge ;)
daten = []
for kategorie, komponenten in kostenarten:
    for name, preis in komponenten:
        daten.append({'Kostenart': kategorie, 'Komponente': name, 'Stückpreis_EUR': preis})

# DataFrame erstellen
kosten_df = pd.DataFrame(daten)


#Kosten

# Produktionskosten berechnen
# Materialkosten & Betriebskosten werden pro Tonne (insgesamt 1000) multipliziert, Entwicklungskosten nur einmalig
kosten_df['Gesamtpreis_EUR'] = kosten_df.apply(
    lambda row: row['Stückpreis_EUR'] * 1000 if row['Kostenart'] == 'Materialkosten' else row['Stückpreis_EUR'],
    axis=1
)

# Betriebskosten werden für 1000 Tonnen berechnet, da sie für 1 Tonne in der Liste stehen
kosten_df['Gesamtpreis_EUR'] = kosten_df.apply(
    lambda row: row['Stückpreis_EUR'] * 1000 if row['Kostenart'] == 'Betriebskosten' else row['Gesamtpreis_EUR'],
    axis=1
)
# Aufsummieren pro Kostenart
gesamt_pro_kategorie = kosten_df.groupby('Kostenart')['Gesamtpreis_EUR'].sum().to_dict()

# Preis ist schon auf 1000 Tonnen hochgerechnet
gesamtkosten_alle_tonnen = kosten_df['Gesamtpreis_EUR'].sum()

# Einmalige Entwicklungskosten (gesamt)
einmalige_entwicklungskosten = gesamt_pro_kategorie.get('Entwicklungskosten', 0)

# Laufende Kosten pro Tonne (Materialkosten + Betriebskosten)
laufende_kosten_pro_tonne = gesamtkosten_alle_tonnen - einmalige_entwicklungskosten
laufende_kosten_pro_tonne /= 1000  # Auf die 1000 Tonnen runterbrechen

# Entwicklungskosten pro Tonne
entwicklungskosten_pro_tonne = einmalige_entwicklungskosten / 1000

# Gesamtpreis pro Tonne
gesamtkosten_pro_tonne = entwicklungskosten_pro_tonne + laufende_kosten_pro_tonne

#Einnahmen & Gewinn

# Laufende Servicegebühren (Subscription-Modell)
monatliche_servicegebuehr = 10.0  # pro Monat und Kunde
einnahmen_durch_service = 12 * monatliche_servicegebuehr * 1000  # 1000 Kunden, aufs Jahr hoch rechnen

# Einnahmen durch Datenverwertung (z.B. Abfalldaten)
einnahmen_durch_daten = 2000  # jährliche Lizenzgebühr

# Einnahmen durch Partnerschaften mit Kommunen oder Entsorgern für Datenlizenzen, idk ob jährlich realistisch
einnahmen_durch_datenpartnerschaften = 10000  

# Einnahmen aus Werbung in der Endanwender App:
werbung_pro_nutzer = 0.10  # €/Nutzer/Monat
anzahl_nutzer = 1000  # bspw. 1000 Nutzer pro Monat
werbung_einnahmen = werbung_pro_nutzer * anzahl_nutzer * 12  # aufs Jahr rechnen

# Einnahmen durch Premium-Servicegebühren für Endanwender (B2C)
monatliche_premium_servicegebuehr_endanwender = 3.0  # pro Nutzer
einnahmen_durch_premium_endanwender = monatliche_premium_servicegebuehr_endanwender * 1000 * 12  # 1000 Endanwender, aufs Jahr

# Einnahmen durch Premium-Servicegebühren für Entsorger (B2B)
monatliche_premium_servicegebuehr_entsorger = 50.0  # pro Tonne
einnahmen_durch_premium_entsorger = monatliche_premium_servicegebuehr_entsorger * 12 * 1000  # aufs Jahr, 1000 Tonnen

# Einnahmen durch Affiliate-Marketing:
kauf_pro_nutzer = 50  # € pro Nutzer/Jahr
provision_pro_nutzer = 0.10  # 10 % Provision
affiliate_einnahmen = kauf_pro_nutzer * provision_pro_nutzer * anzahl_nutzer  # Einnahmen aufs Jahr

# Marge von 40% könnte gering sein, bin mir aber nicht sicher
verkaufspreis_pro_tonne = laufende_kosten_pro_tonne * 1.4
gewinn_pro_tonne = verkaufspreis_pro_tonne - laufende_kosten_pro_tonne

# Gesamtgewinnberechnung
# Gewinn aus Verkauf der 1000 Tonnen -> hier werden die Kosten je Tonne schon abgezogen
gewinn_aus_verkauf = gewinn_pro_tonne * 1000

# Laufende Einnahmen:
# Einnahmen durch Servicegebühren
gewinn_aus_service = einnahmen_durch_service - (5 * 1000)  # Betriebskosten für Service (5 EUR pro Kunde)

# Gesamtgewinn inkl. aller Einnahmequellen
gesamtgewinn = gewinn_aus_verkauf + gewinn_aus_service + einnahmen_durch_daten + einnahmen_durch_datenpartnerschaften + werbung_einnahmen + einnahmen_durch_premium_endanwender + einnahmen_durch_premium_entsorger + affiliate_einnahmen

#Bericht

# Ausgabe
bericht = f"""
=== Kalkulation Smarte Mülltonne ===

Kostenübersicht pro Kategorie:
Entwicklungskosten:
- Softwareentwicklung (App für Endanwender): {gesamt_pro_kategorie.get('Entwicklungskosten', 0):.2f} €
- Softwareentwicklung (App/Dashboard für Entsorger): {gesamt_pro_kategorie.get('Entwicklungskosten', 1):.2f} €
- Technische Entwicklung LoRaWAN (Integration, Kommunikation): {gesamt_pro_kategorie.get('Entwicklungskosten', 2):.2f} €
- Sonstige Entwicklungskosten (Testing, Debugging, Schnittstellen): {gesamt_pro_kategorie.get('Entwicklungskosten', 3):.2f} €
Materialkosten: {gesamt_pro_kategorie.get('Materialkosten', 0):.2f} €
Betriebskosten (jährlich): {gesamt_pro_kategorie.get('Betriebskosten', 0):.2f} €

Einmalige Entwicklungskosten (gesamt): {einmalige_entwicklungskosten:.2f} €
Laufende Kosten pro Tonne (ohne Entwicklungskosten): {laufende_kosten_pro_tonne:.2f} €
Verkaufspreis pro Tonne: {verkaufspreis_pro_tonne:.2f} €
Gewinn pro Tonne: {gewinn_pro_tonne:.2f} €
Gewinn aus Verkauf der 1000 Tonnen: {gewinn_aus_verkauf:.2f} €

Laufende Einnahmen:
Monatliche Servicegebühr (pro Tonne): {monatliche_servicegebuehr:.2f} €
Jährliche Service-Einnahmen (1000 Kunden): {einnahmen_durch_service:.2f} €
Einnahmen durch Datenverwertung: {einnahmen_durch_daten:.2f} €
Einnahmen durch Datenpartnerschaften: {einnahmen_durch_datenpartnerschaften:.2f} €
Werbungseinnahmen aus App: {werbung_einnahmen:.2f} €
Einnahmen durch Premium-Servicegebühren (Endanwender): {einnahmen_durch_premium_endanwender:.2f} €
Einnahmen durch Premium-Servicegebühren (Entsorger): {einnahmen_durch_premium_entsorger:.2f} €
Affiliate-Marketing Einnahmen: {affiliate_einnahmen:.2f} €

Gesamtgewinn aus allen Einnahmequellen: {gesamtgewinn:.2f} €
"""

print(bericht)

# Falls gewünscht, Tabelle als Excel speichern
# kosten_df.to_excel('kalkulation_smarte_muelltone.xlsx', index=False) 




=== Kalkulation Smarte Mülltonne ===

Kostenübersicht pro Kategorie:
Entwicklungskosten:
- Softwareentwicklung (App für Endanwender): 150000.00 €
- Softwareentwicklung (App/Dashboard für Entsorger): 150000.00 €
- Technische Entwicklung LoRaWAN (Integration, Kommunikation): 150000.00 €
- Sonstige Entwicklungskosten (Testing, Debugging, Schnittstellen): 150000.00 €
Materialkosten: 129500.00 €
Betriebskosten (jährlich): 45000.00 €

Einmalige Entwicklungskosten (gesamt): 150000.00 €
Laufende Kosten pro Tonne (ohne Entwicklungskosten): 174.50 €
Verkaufspreis pro Tonne: 244.30 €
Gewinn pro Tonne: 69.80 €
Gewinn aus Verkauf der 1000 Tonnen: 69800.00 €

Laufende Einnahmen:
Monatliche Servicegebühr (pro Tonne): 10.00 €
Jährliche Service-Einnahmen (1000 Kunden): 120000.00 €
Einnahmen durch Datenverwertung: 2000.00 €
Einnahmen durch Datenpartnerschaften: 10000.00 €
Werbungseinnahmen aus App: 1200.00 €
Einnahmen durch Premium-Servicegebühren (Endanwender): 36000.00 €
Einnahmen durch Premium-Servi


=== Kalkulation Smarte Mülltonne ===

Kostenübersicht pro Kategorie:
Entwicklungskosten:
- Softwareentwicklung (App für Endanwender): 150000.00 €
- Softwareentwicklung (App/Dashboard für Entsorger): 150000.00 €
- Technische Entwicklung LoRaWAN (Integration, Kommunikation): 150000.00 €
- Sonstige Entwicklungskosten (Testing, Debugging, Schnittstellen): 150000.00 €
Materialkosten: 129500.00 €
Betriebskosten (jährlich): 45000.00 €

Einmalige Entwicklungskosten (gesamt): 150000.00 €
Laufende Kosten pro Tonne (ohne Entwicklungskosten): 174.50 €
Verkaufspreis pro Tonne: 244.30 €
Gewinn pro Tonne: 69.80 €
Gewinn aus Verkauf der 1000 Tonnen: 69800.00 €

Laufende Einnahmen:
Monatliche Servicegebühr (pro Tonne): 10.00 €
Jährliche Service-Einnahmen (1000 Kunden): 120000.00 €
Einnahmen durch Datenverwertung: 2000.00 €
Einnahmen durch Datenpartnerschaften: 10000.00 €
Werbungseinnahmen aus App: 1200.00 €
Einnahmen durch Premium-Servicegebühren (Endanwender): 36000.00 €
Einnahmen durch Premium-Servi